In [1]:
%load_ext autoreload
%autoreload 2
import torch
from utils import get_mnist_data
from models import ConvNN
from training_and_evaluation import evaluate_robustness_smoothing

# Project 2, part 4: Randomized smoothing certification (20 pt)
In this notebook we compare the robustness of the classifiers from Parts 1-3 via randomized smoothing.

## Your task
Complete the missing code in the respective files, i.e. `models.py`, `training_and_evaluation.py`, `attacks.py`, and this notebook. Make sure that all the functions follow the provided specification, i.e. the output of the function exactly matches the description in the docstring. 

Specifically, for this part you will have to implement the following functions / classes:  
**`training_and_evaluation.py`**:
* `evaluate_robustness_smoothing` (20pt)

## General remarks

Do not add or modify any code outside of the following comment blocks, or where otherwise explicitly stated.
``` python
##########################################################
# YOUR CODE HERE
...
##########################################################
```
After you fill in all the missing code, restart the kernel and re-run all the cells in the notebook.

The following things are **NOT** allowed:
- Using additional `import` statements
- Copying / reusing code from other sources (e.g. code by other students)

If you plagiarise even for a single project task, you won't be eligible for the bonus this semester.

In [2]:
mnist_testset = get_mnist_data(train=False)
use_cuda = torch.cuda.is_available()
model = ConvNN()
if use_cuda:
    model = model.cuda()
    
num_samples_1 = int(1e3)  # reduce this to 1e2 in case it takes too long, e.g. because you don't have CUDA
num_samples_2 = int(1e4)  # reduce this to 1e3 in case it takes too long, e.g. because you don't have CUDA
certification_batch_size = int(5e3)  # reduce this to 5e2 if required (e.g. not enough memory)
sigma = 1
alpha = 0.05

False

In [3]:
training_types = ['standard_training', "adversarial_training", "randomized_smoothing"]

### Robustness certification
Here we first load the checkpoints for the base classifiers of the different training methods of Parts 1-3. Then, perform robustness certification of the smooth classifier via randomized smoothing. For this, you need to implement `evaluate_robustness_smoothing` from `training_and_evaluation.py`. Follow the docstring in that file.

In [4]:
results = {}

for training_type in training_types:
    model.load_state_dict(torch.load(f"models/{training_type}.checkpoint"))
    certification_results = evaluate_robustness_smoothing(model, sigma, mnist_testset, num_samples_1=num_samples_1,
                                                          num_samples_2=num_samples_2, alpha=alpha, 
                                                          certification_batch_size=certification_batch_size)
    results[training_type] = certification_results

100%|██████████| 10000/10000 [08:26<00:00, 19.75it/s]


### Robustness comparison
Compare the robustness of the different training types. As we can see, robust training via randomized smoothing leads to the best robustness.

Note that the number of certified points will be lower in case you had to reduce the number of samples for performance reasons.

In [5]:
for k,v in results.items():
    print(f"{k}: correct_certified {v['correct_certified']}, avg. certifiable radius: {v['avg_radius']}")


standard_training: correct_certified 2049, avg. certifiable radius: 0.7818142620224044, avg. radius: 0.6416349648417873, avg. corr certifiable radius: 1.1656302493885478,
adversarial_training: correct_certified 944, avg. certifiable radius: 0.46022926207128323, avg. radius: 0.3535020961969527, avg. corr certifiable radius: 0.640199294330325,
randomized_smoothing: correct_certified 7434, avg. certifiable radius: 0.9767335592188942, avg. radius: 0.7549173679202833, avg. corr certifiable radius: 1.0033987571195686,


{'standard_training': {'abstains': 1793,
  'false_predictions': 6158,
  'correct_certified': 2049,
  'avg_radius': 1.1656302493885478,
  'avg_radius_2': 0.7818142620224044,
  'avg_radius_all': 0.6416349648417873},
 'adversarial_training': {'abstains': 2319,
  'false_predictions': 6737,
  'correct_certified': 944,
  'avg_radius': 0.640199294330325,
  'avg_radius_2': 0.46022926207128323,
  'avg_radius_all': 0.3535020961969527},
 'randomized_smoothing': {'abstains': 2271,
  'false_predictions': 295,
  'correct_certified': 7434,
  'avg_radius': 1.0033987571195686,
  'avg_radius_2': 0.9767335592188942,
  'avg_radius_all': 0.7549173679202833}}